In [60]:
# Importing all the required modules
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import randint, uniform
from pandas.plotting import scatter_matrix

# Libraries related to outlier detection
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Encoding and Standardization
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler

# Ignore all warnings

import warnings
warnings.filterwarnings("ignore")

# Set the default figure size for seaborn plots to 11x8 inches, ensuring all plots have a consistent, larger appearance
sns.set(rc={'figure.figsize':(11,8)})

# Format all floating-point numbers in pandas DataFrames to display with two decimal places,
# providing a cleaner, more readable presentation of numerical data
pd.options.display.float_format = '{:.2f}'.format

from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer


In [5]:
# Load the CSV and Excel files into dataframes
file_2015 = "15tstcar.csv"
file_2016 = "16tstcar.csv"
file_2017 = "17tstcar-2018-05-30.xlsx"
file_2018 = "18tstcar-2018-10-24.xlsx"
file_2019 = "19tstcar-2020-10-02.xlsx"
file_2020 = "20tstcar-2021-03-02.xlsx"
file_2021 = "21-tstcar-2022-04-15.xlsx"
file_2022 = "22-testcar-2023-06-13.xlsx"
file_2023 = "23-testcar-2024-05-17_0.xlsx"
file_2024 = "24-testcar-2024-05-17_0 (1).xlsx"

In [6]:
# Load CSV files
df_2015 = pd.read_csv(file_2015)
df_2016 = pd.read_csv(file_2016)

# Load Excel files
df_2017 = pd.read_excel(file_2017)
df_2018 = pd.read_excel(file_2018)
df_2019 = pd.read_excel(file_2019)
df_2020 = pd.read_excel(file_2020)
df_2021 = pd.read_excel(file_2021)
df_2022 = pd.read_excel(file_2022)
df_2023 = pd.read_excel(file_2023)
df_2024 = pd.read_excel(file_2024)

In [7]:
df = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)
df.shape

(45479, 67)

### Step 1: Data preprocessing and Exploration

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45479 entries, 0 to 45478
Data columns (total 67 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Model Year                      45479 non-null  int64  
 1   Vehicle Manufacturer Name       45479 non-null  object 
 2   Veh Mfr Code                    45479 non-null  object 
 3   Represented Test Veh Make       45479 non-null  object 
 4   Represented Test Veh Model      45479 non-null  object 
 5   Test Vehicle ID                 45479 non-null  object 
 6   Test Veh Configuration #        45479 non-null  int64  
 7   Test Veh Displacement (L)       45479 non-null  float64
 8   Actual Tested Testgroup         45479 non-null  object 
 9   Vehicle Type                    45479 non-null  object 
 10  Rated Horsepower                45479 non-null  int64  
 11  # of Cylinders and Rotors       43103 non-null  float64
 12  Engine Code                     

Check for missing values

In [9]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    total_missing = missing_values.sum()
    total_rows = len(df)

    if total_missing > 0:
        print("Missing values found in the following columns (sorted by missing count):")
        # Calculate the percentage of missing values
        missing_percentage = round(((missing_values / total_rows) * 100), 2)
        # Combine missing counts and percentages
        data_types = df.dtypes
        missing_info = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage, 'Data Type': data_types})
        # Filter and sort by missing values
        sorted_missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False)
        #print(sorted_missing_info)
        return sorted_missing_info
    else:
        print("No missing values found in the dataframe.")


In [10]:
check_missing_values(df)

Missing values found in the following columns (sorted by missing count):


,Missing Values,Percentage (%),Data Type
FE Bag 4,43715,96.12,float64
Averaging Weighting Factor,43686,96.06,float64
Averaging Group ID,43686,96.06,object
ADFE Test Number,40746,89.59,object
ADFE Total Road Load HP,40746,89.59,float64
ADFE N/V Ratio,40746,89.59,float64
ADFE Equiv. Test Weight (lbs.),40742,89.58,float64
PM (g/mi),38232,84.07,float64
FE Bag 3,27793,61.11,float64
FE Bag 1,24410,53.67,float64


There are no discrepancies with regard to data types of the various columns across the datasets. However, there are several attributes with significant number of null values in the dataset. Depending on the importance of the variable, we either perform imputation or remove the attribute from the list of predictor variables.

From the above list, we remove the attributes with greater than 50% of the values missing (ADFE Test Number, ADFE Total Road Load HP, ADFE N/V Ratio, ADFE Equiv.Test Weight, PM(g/mi), FE Bag 3, FE Bag 1, FE Bag 2 and N2O (g/mi)). 

In [11]:
def drop_columns_with_missing_values(df, columns_to_drop):
    df_reduced = df.copy()
    
    # Drop the specified columns
    df_reduced = df_reduced.drop(columns=columns_to_drop, errors='ignore')
    
    return df_reduced

In [12]:
columns_to_remove = [
    'FE Bag 4', 'Averaging Weighting Factor', 'Averaging Group ID', 
    'ADFE Test Number', 'ADFE Total Road Load HP', 'ADFE N/V Ratio', 
    'ADFE Equiv. Test Weight (lbs.)', 'PM (g/mi)', 'FE Bag 3', 
    'FE Bag 1', 'FE Bag 2', 'N2O (g/mi)'
]
df_reduced = drop_columns_with_missing_values(df, columns_to_remove)

In [13]:
check_missing_values(df_reduced)

Missing values found in the following columns (sorted by missing count):


,Missing Values,Percentage (%),Data Type
DT-Inertia Work Ratio Rating,9389,20.64,float64
DT-Absolute Speed Change Ratg,9389,20.64,float64
DT-Energy Economy Rating,9389,20.64,float64
CH4 (g/mi),8816,19.38,float64
NOx (g/mi),6266,13.78,float64
THC (g/mi),6008,13.21,float64
CO (g/mi),5902,12.98,float64
CO2 (g/mi),2913,6.41,float64
Aftertreatment Device Cd,2444,5.37,object
Aftertreatment Device Desc,2444,5.37,object


In [15]:
## New Mode Imputation

def impute_data(df, knn_neighbors=7):
    # Create a copy of the DataFrame to avoid changing the original
    df_imputed = df.copy()
    
    # Separate numeric and categorical columns
    numeric_cols = df_imputed.select_dtypes(include=['number']).columns
    categorical_cols = [
        'Vehicle Manufacturer Name', 
        'Vehicle Type', 
        'Tested Transmission Type Code', 
        'Transmission Lockup?', 
        'Drive System Code', 
        'Transmission Overdrive Desc', 
        'Shift Indicator Light Use Desc', 
        'Test Procedure Description', 
        'Test Fuel Type Description', 
        'Test Category', 
        'Aftertreatment Device Cd',
        'Aftertreatment Device Desc',  # Explicitly add missing categorical cols
        'Engine Code'
    ]

    # 1. KNN Imputation for numeric columns
    knn_imputer = KNNImputer(n_neighbors=knn_neighbors)
    df_imputed[numeric_cols] = pd.DataFrame(knn_imputer.fit_transform(df_imputed[numeric_cols]), 
                                            columns=numeric_cols, index=df_imputed.index)

    # 2. Encode categorical columns as integers and replace missing values with np.nan
    categorical_codes = {}
    for col in categorical_cols:
        df_imputed[col] = pd.Categorical(df_imputed[col]).codes
        df_imputed[col].replace(-1, np.nan, inplace=True)  # Replace -1 with np.nan for missing values
        categorical_codes[col] = pd.Categorical(df[col].dropna()).categories  # Save original categories

    # Apply MICE imputer to encoded categorical columns
    mice_imputer = IterativeImputer(max_iter=10, random_state=42, sample_posterior=True)
    try:
        imputed_categorical = pd.DataFrame(mice_imputer.fit_transform(df_imputed[categorical_cols]), 
                                           columns=categorical_cols, index=df_imputed.index)
    except Exception as e:
        print("MICE imputation failed for categorical columns:", e)
        # Fallback: Simple mode imputation
        simple_imputer = SimpleImputer(strategy='most_frequent')
        imputed_categorical = pd.DataFrame(simple_imputer.fit_transform(df_imputed[categorical_cols]), 
                                           columns=categorical_cols, index=df_imputed.index)

    # Round imputed values to nearest integer, clip to valid range, and reconvert to categories
    for col in categorical_cols:
        categories = categorical_codes[col]
        imputed_categorical[col] = imputed_categorical[col].round().clip(0, len(categories) - 1).astype(int)
        df_imputed[col] = pd.Categorical.from_codes(imputed_categorical[col], categories=categories)

    return df_imputed


In [16]:
df_imputed = impute_data(df_reduced)

In [17]:
check_missing_values(df_imputed)

No missing values found in the dataframe.


In [18]:
categorical_cols = df_reduced.select_dtypes(include=['object']).columns
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"Column '{col}' has {unique_count} unique values.")

Column 'Vehicle Manufacturer Name' has 55 unique values.
Column 'Veh Mfr Code' has 54 unique values.
Column 'Represented Test Veh Make' has 97 unique values.
Column 'Represented Test Veh Model' has 2326 unique values.
Column 'Test Vehicle ID' has 4366 unique values.
Column 'Actual Tested Testgroup' has 2432 unique values.
Column 'Vehicle Type' has 3 unique values.
Column 'Engine Code' has 2031 unique values.
Column 'Tested Transmission Type Code' has 8 unique values.
Column 'Tested Transmission Type' has 8 unique values.
Column 'Transmission Lockup?' has 2 unique values.
Column 'Drive System Code' has 5 unique values.
Column 'Drive System Description' has 5 unique values.
Column 'Transmission Overdrive Desc' has 3 unique values.
Column 'Shift Indicator Light Use Desc' has 4 unique values.
Column 'Test Number' has 23168 unique values.
Column 'Test Originator' has 2 unique values.
Column 'Analytically Derived FE?' has 2 unique values.
Column 'Test Procedure Description' has 16 unique val

Among the categorical variables, some of them have extremely large number of values such as Represented Test Veh Model, Test Vehicle ID, Actual Tested Testgroup and Engine Code. Choosing these values will create 

When selecting categorical variables, it is important to consider its relevance to the response variable as well as the number of unique values. Some of the categorical attributes such as Represented Test Veh Model, Test Vehicle ID, Actual Tested Testgroup and Engine Code have an extremely large number of values. Encoding these values will lead to issues with model interpretability and complexity. Taking these factors into account we choose the following categorical variables - **Vehicle Manufacturer Name, Vehicle Type, TTested Transmission Type Code, Transmission Lockup?, Drive System Code, Transmission Overdrive Desc, Shift Indicator Light Use Desc, Test Procedure Description, Test Fuel Type Description, Test Category and Aftertreatment Device Cd**

In [19]:
df_imputed.describe()

,Model Year,Test Veh Configuration #,Test Veh Displacement (L),Rated Horsepower,# of Cylinders and Rotors,# of Gears,Transmission Overdrive Code,Equivalent Test Weight (lbs.),Axle Ratio,N/V Ratio,...,RND_ADJ_FE,DT-Inertia Work Ratio Rating,DT-Absolute Speed Change Ratg,DT-Energy Economy Rating,Target Coef A (lbf),Target Coef B (lbf/mph),Target Coef C (lbf/mph**2),Set Coef A (lbf),Set Coef B (lbf/mph),Set Coef C (lbf/mph**2)
count,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,...,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00,45479.00
mean,2019.39,1.18,3.39,298.05,5.44,6.28,1.95,4396.41,3.62,32.95,...,49.98,8.00,7.90,7.45,39.26,0.23,0.02,11.09,0.11,0.02
std,2.87,2.03,7.18,151.48,1.82,2.72,0.21,858.87,1.15,46.05,...,363.58,24.67,24.61,24.85,11.94,0.32,0.01,11.29,0.30,0.09
min,2015.00,0.00,0.00,1.00,2.00,1.00,1.00,2125.00,0.00,0.00,...,0.00,-29.17,-24.01,-17.82,9.87,-0.85,0.00,-99.90,-1.98,-0.09
25%,2017.00,0.00,2.00,183.00,4.00,6.00,2.00,3750.00,3.14,24.30,...,24.00,-0.62,-0.51,-0.57,30.62,0.05,0.02,5.26,-0.02,0.02
50%,2019.00,0.00,2.50,275.00,4.86,7.00,2.00,4250.00,3.42,26.90,...,31.10,0.62,0.41,-0.10,38.10,0.25,0.02,11.40,0.11,0.02
75%,2022.00,1.00,3.50,362.00,6.00,8.00,2.00,5000.00,3.77,31.20,...,41.10,2.70,1.99,0.39,46.70,0.42,0.03,17.79,0.24,0.03
max,2024.00,25.00,100.00,1839.00,16.00,10.00,2.00,9000.00,9.99,999.90,...,10000.00,99.99,99.99,99.99,150.00,2.41,0.22,150.20,10.00,10.00


### Performing encoding of selected categorical variables

Encoding categorical attributes with high cardinality values can increase the complexity and lead to model interpretability issues. For example, Vehicle Manufacturer Name attribute has 55 unique values. So, performing one-hot encoding will result in the addition of 55 new columns to the dataset. In order to prevent that, we will first re-categorize the various vehicle manufacturer names based on manufacturer groups and then perform one-hot encoding. 

In [20]:
brand_groups = {
    'Volkswagen Group': ['Audi', 'Bentley', 'Bugatti', 'Volkswagen', 'Porsche', 'Lamborghini'],
    'FCA Group': ['FCA Italy', 'FCA US LLC', 'Ferrari', 'Maserati'],
    'Ford Motor Company': ['FOMOCO'],
    'General Motors': ['GM'],
    'Hyundai Motor Group': ['Hyundai', 'Kia', 'Genesis'],
    'BMW Group': ['BMW', 'Mini', 'Rolls-Royce'],
    'Toyota Group': ['Toyota', 'Lexus'],
    'Daimler AG': ['Mercedes-Benz'],
    'Honda': ['Honda', 'Acura'],
    'Tata Motors': ['Jaguar Land Rover L'],
    'Electric Vehicle Manufacturers': ['Tesla', 'Rivian Automotive L', 'Lucid USA, Inc', 'Fisker Group Inc.'],
    'High-Performance and Specialty': [
        'Aston Martin', 'McLaren Automotive', 'Pagani Automobili S', 'Koenigsegg', 
        'Rimac Automobili', 'Maserati', 'Bugatti Rimac LLC', 'RUF'
    ],
    'Asian Manufacturers': ['BYD Motors Inc.', 'Nissan', 'Mazda', 'Mitsubishi Motors Co', 'Subaru'],
    'Other': [
        'Quantum Fuel System', 'CSC', 'Mobility Ventures L', 'Bluecar, SAS', 
        'Ineos Automotive Li', 'Lordstown EV Corpor', 'Canoo Technologies', 
        'Faraday&Future Inc.', 'Vinfast Trading and', 'Mullen Automotive'
    ]
}

def map_manufacturer_group(df, column, brand_groups):
    # Create a reverse mapping dictionary from brand_groups
    group_mapping = {manufacturer: group for group, manufacturers in brand_groups.items() for manufacturer in manufacturers}
    
    # Map the column values to their respective groups
    df[column] = df[column].map(group_mapping) 
    
    return df

In [21]:
# Usage
df_vmn = map_manufacturer_group(df_imputed, 'Vehicle Manufacturer Name', brand_groups)

In [22]:
def label_encode_df(df):
    # Create a copy of the DataFrame to avoid modifying the original
    df_encoded = df.copy()
    
    # Loop through all categorical columns and apply label encoding
    for col in df_encoded.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))  # Convert to string in case of mixed types
    
    return df_encoded

In [23]:
df_encoded = label_encode_df(df_vmn)

In [24]:
len(df_encoded.columns)

55

### Removing multivariate outliers using a combination of three techniques (KNN, Decision Tree and Mahalanobis distance)

In [25]:
def removing_outliers(df, selected_columns):
    # Keep the other columns intact
    other_columns = df.drop(columns=selected_columns)
    
    # Select only the specified columns for outlier detection
    df_num = df[selected_columns]
    
    # Row count before removing outliers
    before_count = len(df_num)
    
    # Local Outlier Factor with lower contamination (1-2%)
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
    y_pred_lof = lof.fit_predict(df_num)
    
    # Isolation Forest with lower contamination (1-2%)
    iforest = IsolationForest(n_estimators=100, contamination=0.01)
    y_pred_if = iforest.fit_predict(df_num)
    
    # Elliptic Envelope with lower contamination (1-2%)
    rob_cov = EllipticEnvelope(contamination=0.01)
    rob_cov.fit(df_num)
    y_pred_rob = rob_cov.predict(df_num)
    
    # Assign outlier labels
    df_num["y_pred_lof"] = y_pred_lof
    df_num["y_pred_if"] = y_pred_if
    df_num["y_pred_rob"] = y_pred_rob
    
    df_num["y_pred_lof_2"] = np.where(df_num["y_pred_lof"] < 0, -1, 0)
    df_num["y_pred_if_2"] = np.where(df_num["y_pred_if"] < 0, -1, 0)
    df_num["y_pred_rob_2"] = np.where(df_num["y_pred_rob"] < 0, -1, 0)
    
    # Crosstab for checking consistency (optional for debugging)
    pd.crosstab(df_num["y_pred_if_2"], df_num["y_pred_rob_2"])
    
    # Sum of outlier labels from all methods
    df_num["all_out"] = df_num.loc[:, ["y_pred_if_2", "y_pred_rob_2", "y_pred_lof_2"]].sum(axis=1)
    
    # Filter out outliers
    df_num_filtered = df_num[df_num["all_out"] > -1]
    
    # Row count after removing outliers
    after_count = len(df_num_filtered)
    
    # Calculate and print the percentage of rows removed
    percentage_removed = ((before_count - after_count) / before_count) * 100
    print(f"Percentage of rows removed as outliers: {percentage_removed:.2f}%")
    
    # Drop outlier-related columns used for filtering
    df_num_filtered = df_num_filtered.drop(columns=["y_pred_lof", "y_pred_if", "y_pred_rob", "y_pred_lof_2", "y_pred_if_2", "y_pred_rob_2", "all_out"])
    
    # Concatenate the filtered numerical columns with the other non-imputed columns
    final_df = pd.concat([other_columns.reset_index(drop=True), df_num_filtered.reset_index(drop=True)], axis=1)
    
    return final_df

In [26]:
df_outliers_removed = removing_outliers(df_encoded, df_imputed.select_dtypes(include=['object']).columns.tolist())

Percentage of rows removed as outliers: 2.95%


In [27]:
df_outliers_removed.columns

Index(['Model Year', 'Test Veh Configuration #', 'Test Veh Displacement (L)',
       'Vehicle Type', 'Rated Horsepower', '# of Cylinders and Rotors',
       'Engine Code', 'Tested Transmission Type Code', '# of Gears',
       'Transmission Lockup?', 'Drive System Code',
       'Transmission Overdrive Code', 'Transmission Overdrive Desc',
       'Equivalent Test Weight (lbs.)', 'Axle Ratio', 'N/V Ratio',
       'Shift Indicator Light Use Cd', 'Shift Indicator Light Use Desc',
       'Test Procedure Cd', 'Test Procedure Description', 'Test Fuel Type Cd',
       'Test Fuel Type Description', 'Test Category', 'THC (g/mi)',
       'CO (g/mi)', 'CO2 (g/mi)', 'NOx (g/mi)', 'CH4 (g/mi)', 'RND_ADJ_FE',
       'DT-Inertia Work Ratio Rating', 'DT-Absolute Speed Change Ratg',
       'DT-Energy Economy Rating', 'Target Coef A (lbf)',
       'Target Coef B (lbf/mph)', 'Target Coef C (lbf/mph**2)',
       'Set Coef A (lbf)', 'Set Coef B (lbf/mph)', 'Set Coef C (lbf/mph**2)',
       'Aftertreatment De

### Performing standardization 

In [28]:

def standardize_numerical_columns(df):
    # Select only numerical columns
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_standardized = df.copy()
    
    # Initialize the StandardScaler
    scaler = RobustScaler()
    
    # Fit and transform only the numerical columns
    df_standardized[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df_standardized

# Usage
df_standardized = standardize_numerical_columns(df_outliers_removed)

We select several vehicle specific characteristics, emission specific factors along with factors related to drive and transmission for predicting CO emission and attributes related to Engine and Powertrain, Aerodynamics and Resistance, Vehicle load and Weight for  Miles per Gallon prediction.

In [29]:
# Attributes for predicting miles per gallon (RND_ADJ_FE)
mpg_attributes = [
    'Vehicle Manufacturer Name', 'Rated Horsepower', '# of Cylinders and Rotors',
    'Test Veh Displacement (L)', 'Transmission Overdrive Code', '# of Gears', 'Axle Ratio',
    'Target Coef A (lbf)', 'Target Coef B (lbf/mph)', 'Target Coef C (lbf/mph**2)',
    'Set Coef A (lbf)', 'Set Coef B (lbf/mph)', 'Set Coef C (lbf/mph**2)',
    'Equivalent Test Weight (lbs.)', 'DT-Inertia Work Ratio Rating', 'RND_ADJ_FE'
]

# Attributes for predicting CO emissions (CO (g/mi))
co_emission_attributes = [
    'Vehicle Manufacturer Name', 'Vehicle Type', 'Rated Horsepower', '# of Cylinders and Rotors',
    'Test Veh Displacement (L)', 'Axle Ratio', 'N/V Ratio', 'THC (g/mi)', 'NOx (g/mi)',
    'CH4 (g/mi)', 'Aftertreatment Device Desc', 'Test Fuel Type Description',
    'Drive System Description', 'Transmission Overdrive Desc', 'Equivalent Test Weight (lbs.)',
    'Shift Indicator Light Use Desc', 'CO (g/mi)'
]

Data Preparation for training models

In [ ]:
df_mpg = df_standardized[mpg_attributes].dropna()

# Separate features and target
X_mpg = df_mpg.drop(columns='RND_ADJ_FE')
y_mpg = df_mpg['RND_ADJ_FE']

# One-hot encode categorical features
X_mpg = pd.get_dummies(X_mpg, drop_first=True)

# Split data into training and testing sets
X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg = train_test_split(X_mpg, y_mpg, test_size=0.2, random_state=42)

In [ ]:
df_co = df_standardized[co_emission_attributes].dropna()

# Separate features and target
X_co = df_co.drop(columns='CO (g/mi)')
y_co = df_co['CO (g/mi)']

# One-hot encode categorical features
X_co = pd.get_dummies(X_co, drop_first=True)

# Split data into training and testing sets
X_train_co, X_test_co, y_train_co, y_test_co = train_test_split(X_co, y_co, test_size=0.2, random_state=42)


### Multiple Linear Regression Model

In [ ]:

def multipleLinearRegression(X_train,X_test,y_train,y_test,pred):
    # Step 1: Train MLR model
    mlr_mpg = LinearRegression()
    mlr_mpg.fit(X_train, y_train)

    # Step 2: Predict and evaluate the model
    y_pred = mlr_mpg.predict(X_test)
    print("Performance for " + pred + " Prediction:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))


In [37]:
multipleLinearRegression(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg,"RND_ADJ_FE")

Performance for RND_ADJ_FE Prediction:
MAE: 2.4342762715581854
MSE: 535.9374547081833
R-squared: 0.009192062225476971


In [38]:
multipleLinearRegression(X_train_co, X_test_co, y_train_co, y_test_co,"CO (g/mi)")

Performance for CO (g/mi) Prediction:
MAE: 1.0289465362094243
MSE: 238.6408636402308
R-squared: 0.00417384054404879


### K-Nearest Neighbors (KNN)

In [ ]:

def KNN(X_train,X_test,y_train,y_test,pred):
    # Step 1: Train KNN model
    knn_mpg = KNeighborsRegressor(n_neighbors=5)
    knn_mpg.fit(X_train, y_train)

    # Step 2: Predict and evaluate the model
    y_pred = knn_mpg.predict(X_test)
    print("Performance for RND_ADJ_FE Prediction using KNN:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))



In [42]:
KNN(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg,"RND_ADJ_FE")

Performance for RND_ADJ_FE Prediction using KNN:
MAE: 1.1259577920709303
MSE: 287.1068665142897
R-squared: 0.4692146260111253


In [43]:
KNN(X_train_co, X_test_co, y_train_co, y_test_co,"CO (g/mi)")

Performance for RND_ADJ_FE Prediction using KNN:
MAE: 0.6424925271803955
MSE: 200.43704643135186
R-squared: 0.16359482145798754


### K-Nearest Neighbors with grid search (KNN)

In [ ]:

def KNNGridSearch(X_train,X_test,y_train,y_test,pred):

# Define the hyperparameters grid to tune
    knn_params = {
        'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    }

    # Step 1: Perform Grid Search
    knn = KNeighborsRegressor()
    grid_search_mpg = GridSearchCV(knn, knn_params, cv=5, scoring='r2', n_jobs=-1)
    grid_search_mpg.fit(X_train, y_train)

    # Best model for RND_ADJ_FE
    best_knn_mpg = grid_search_mpg.best_estimator_

    # Step 2: Predict and evaluate the best KNN model
    y_pred = best_knn_mpg.predict(X_test)
    print("Best Hyperparameters for " + pred + " Prediction:", grid_search_mpg.best_params_)
    print("Performance for " + pred + " Prediction using Best KNN:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))


In [45]:
KNNGridSearch(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg,"RND_ADJ_FE")

Best Hyperparameters for RND_ADJ_FE Prediction: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Performance for RND_ADJ_FE Prediction using Best KNN:
MAE: 0.4834158939914406
MSE: 127.091893939184
R-squared: 0.7650403862698746


In [46]:
KNNGridSearch(X_train_co, X_test_co, y_train_co, y_test_co,"CO (g/mi)")

Best Hyperparameters for CO (g/mi) Prediction: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Performance for CO (g/mi) Prediction using Best KNN:
MAE: 0.38041678324861994
MSE: 109.38440866028965
R-squared: 0.5435490220788275


### Random Forest

In [ ]:

def randomForest(X_train,X_test,y_train,y_test,pred): 
    # Step 1: Train Random Forest model
    rf_mpg = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_mpg.fit(X_train, y_train)

    # Step 2: Predict and evaluate the model 
    y_pred = rf_mpg.predict(X_test)
    print("Performance for " + pred + " Prediction using Random Forest:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))

In [48]:
randomForest(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg,"RND_ADJ_FE")

Performance for RND_ADJ_FE Prediction using Random Forest:
MAE: 0.6881253699849211
MSE: 166.2662820125377
R-squared: 0.6926172064388014


In [49]:
randomForest(X_train_co, X_test_co, y_train_co, y_test_co,"CO (g/mi)")

Performance for CO (g/mi) Prediction using Random Forest:
MAE: 0.3181171325666876
MSE: 25.99663841756402
R-squared: 0.8915184424024037


### Random Forest with grid search

In [ ]:

def randomForestGridSearch(X_train,X_test,y_train,y_test,pred): 
    # Define the hyperparameters grid to tune
    rf_params = {
        'n_estimators': [50, 100],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }

    # Step 1: Perform Grid Search
    rf = RandomForestRegressor(random_state=42)
    grid_search_rf = GridSearchCV(rf, rf_params, cv=5, scoring='r2', n_jobs=-1)
    grid_search_rf.fit(X_train, y_train)

    # Best model
    best_rf_co = grid_search_rf.best_estimator_

    # Step 2: Predict and evaluate the best Random Forest model
    y_pred = best_rf_co.predict(X_test)
    print("Best Hyperparameters for " + pred + " Emissions Prediction:", grid_search_rf.best_params_)
    print("Performance for " + pred + " Emissions Prediction using Best Random Forest:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))


In [52]:
randomForestGridSearch(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg,"RND_ADJ_FE")

Best Hyperparameters for RND_ADJ_FE Emissions Prediction: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Performance for RND_ADJ_FE Emissions Prediction using Best Random Forest:
MAE: 0.7484232906097059
MSE: 171.12372167345143
R-squared: 0.6836370731582964


In [54]:
randomForestGridSearch(X_train_co, X_test_co, y_train_co, y_test_co,"CO (g/mi)")

Best Hyperparameters for CO (g/mi) Emissions Prediction: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Performance for CO (g/mi) Emissions Prediction using Best Random Forest:
MAE: 0.36264782429864895
MSE: 26.429427482918765
R-squared: 0.8897124538293131


### XGBoost

In [61]:

# Define the hyperparameters grid to tune
xgb_params = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5)
}

def train_xgboost_model(X, y, model_name):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the XGBoost model
    xgb_model = XGBRegressor(random_state=42)

    # Initialize RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(xgb_model, xgb_params, n_iter=10, cv=5, scoring='r2', random_state=42, n_jobs=-1)
    
    # Perform hyperparameter tuning
    print(f"Training XGBoost model for {model_name} prediction...")
    random_search.fit(X_train, y_train)
    
    # Best model with optimal hyperparameters
    best_xgb_model = random_search.best_estimator_
    print(f"\nBest Parameters for {model_name}:", random_search.best_params_)

    # Evaluate the model
    y_pred = best_xgb_model.predict(X_test)
    print(f"\nPerformance for {model_name} Prediction using XGBoost:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))

    return best_xgb_model

# Prepare the data for RND_ADJ_FE (mileage) prediction
mpg_features = [
    'Rated Horsepower', '# of Cylinders and Rotors', 'Test Veh Displacement (L)', '# of Gears', 
    'Axle Ratio', 'Target Coef A (lbf)', 'Target Coef B (lbf/mph)', 'Target Coef C (lbf/mph**2)', 
    'Set Coef A (lbf)', 'Set Coef B (lbf/mph)', 'Set Coef C (lbf/mph**2)', 
    'Equivalent Test Weight (lbs.)', 'DT-Inertia Work Ratio Rating'
]

df_mpg = df_standardized[mpg_features + ['RND_ADJ_FE']].dropna()
X_mpg = df_mpg[mpg_features]
y_mpg = df_mpg['RND_ADJ_FE']

# Train XGBoost model for RND_ADJ_FE prediction
best_xgb_mpg_model = train_xgboost_model(X_mpg, y_mpg, "RND_ADJ_FE (mileage)")

# Prepare the data for CO emissions prediction
co_features = [
    'Rated Horsepower', '# of Cylinders and Rotors', 'Test Veh Displacement (L)', 'Axle Ratio', 
    'N/V Ratio', 'THC (g/mi)', 'NOx (g/mi)', 'CH4 (g/mi)', 'Equivalent Test Weight (lbs.)'
]

df_co = df_standardized[co_features + ['CO (g/mi)']].dropna()
X_co = df_co[co_features]
y_co = df_co['CO (g/mi)']

# Train XGBoost model for CO emissions prediction
best_xgb_co_model = train_xgboost_model(X_co, y_co, "CO emissions")

Training XGBoost model for RND_ADJ_FE (mileage) prediction...

Best Parameters for RND_ADJ_FE (mileage): {'colsample_bytree': 0.8925879806965068, 'learning_rate': 0.04993475643167195, 'max_depth': 9, 'n_estimators': 113, 'subsample': 0.73338144662399}

Performance for RND_ADJ_FE (mileage) Prediction using XGBoost:
MAE: 0.7233477944258316
MSE: 51.72043461006872
R-squared: 0.867868394174327
Training XGBoost model for CO emissions prediction...

Best Parameters for CO emissions: {'colsample_bytree': 0.8421165132560784, 'learning_rate': 0.09803049874792026, 'max_depth': 9, 'n_estimators': 57, 'subsample': 0.5171942605576092}

Performance for CO emissions Prediction using XGBoost:
MAE: 0.48720330442268955
MSE: 14.198449587697246
R-squared: 0.9311217572221294


### Gradient Boosted Trees

In [65]:
 
def gradientBoostedTree(X_train, X_test, y_train, y_test):
    # Suppress warning messages
    warnings.filterwarnings('ignore')
 
 
    # Define the Gradient Boosting model
    gb_model = GradientBoostingRegressor(random_state=42)
 
    # Define the hyperparameter grid
    param_grid = {
        'n_estimators': randint(50, 200, 500),
        'max_depth': randint(3,5, 20),
        'learning_rate': [0.01, 0.001,0.1,  ]
    }
 
    # Initialize RandomizedSearchCV
    random_search = RandomizedSearchCV(gb_model, param_grid, n_iter=10, cv=5, scoring='r2', random_state=42, n_jobs=-1, return_train_score=True, error_score='raise')
 
    # Perform hyperparameter tuning
    print("Training Gradient Boosting...")
    random_search.fit(X_train, y_train)
 
    # Display the best parameters
    best_params = random_search.best_params_
    print("\nBest Parameters:", best_params)
 
    # Train the Gradient Boosting model with the best parameters
    best_gb_model = GradientBoostingRegressor(random_state=42, **best_params)
    print("Hyperparameters of the Best Model:", best_gb_model.get_params())
    best_gb_model.fit(X_train, y_train)
 
    # Evaluate the model
    y_pred = best_gb_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print("R-2 Score:", r2)
 
    # Save the best model
    with open('best_gb_model.pkl', 'wb') as f:
        pickle.dump(best_gb_model, f)
    print("Best Gradient Boosting model saved as 'best_gb_model.pkl'")
 
    # Load the model from disk
    with open('best_gb_model.pkl', 'rb') as f:
        loaded_gb_model = pickle.load(f)
 
    # Apply the loaded model on a new sample
    new_sample = X_test.iloc[0, :].values.reshape(1, -1)
    prediction = loaded_gb_model.predict(new_sample)
    print("\nPrediction for a new sample:", prediction)
 
    # Save all R-2 values and parameter combinations
    cv_results_df = pd.DataFrame(random_search.cv_results_)
    cv_results_df = cv_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
    print("\nAll R-2 values and parameter combinations:")
    print(cv_results_df)


In [66]:
gradientBoostedTree(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg)

Training Gradient Boosting...

Best Parameters: {'learning_rate': 0.001, 'max_depth': 23, 'n_estimators': 638}
Hyperparameters of the Best Model: {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.001, 'loss': 'squared_error', 'max_depth': 23, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 638, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
R-2 Score: 0.6778552906674554
Best Gradient Boosting model saved as 'best_gb_model.pkl'

Prediction for a new sample: [0.13526347]

All R-2 values and parameter combinations:
                                              params  mean_test_score  \
0  {'learning_rate': 0.1, 'max_depth': 24, 'n_est...             0.41   
1  {'learning_rate': 0.1, 'max_depth': 23, 'n_est...             0.41   

In [67]:
gradientBoostedTree(X_train_co, X_test_co, y_train_co, y_test_co)

Training Gradient Boosting...

Best Parameters: {'learning_rate': 0.001, 'max_depth': 24, 'n_estimators': 587}
Hyperparameters of the Best Model: {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.001, 'loss': 'squared_error', 'max_depth': 24, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 587, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
R-2 Score: 0.6867532308189546
Best Gradient Boosting model saved as 'best_gb_model.pkl'

Prediction for a new sample: [1.05831353]

All R-2 values and parameter combinations:
                                              params  mean_test_score  \
0  {'learning_rate': 0.1, 'max_depth': 24, 'n_est...             0.49   
1  {'learning_rate': 0.1, 'max_depth': 23, 'n_est...             0.08   

### CatBoost

In [59]:
def catBoost(X_train, X_test, y_train, y_test, model_name):

    # Define the hyperparameters grid to tune
    catboost_params = {
        'iterations': randint(50, 200),
        'depth': randint(3, 10),
        'learning_rate': uniform(0.01, 0.2),
        'l2_leaf_reg': uniform(1, 10),
        'subsample': uniform(0.5, 0.5)
    }

    # Initialize the CatBoost model
    cat_model = CatBoostRegressor(silent=True, random_state=42)

    # Initialize RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(cat_model, catboost_params, n_iter=10, cv=5, scoring='r2', random_state=42, n_jobs=-1)
    
    # Perform hyperparameter tuning
    print(f"Training CatBoost model for {model_name} prediction...")
    random_search.fit(X_train, y_train)
    
    # Best model with optimal hyperparameters
    best_cat_model = random_search.best_estimator_
    print(f"\nBest Parameters for {model_name}:", random_search.best_params_)

    # Evaluate the model
    y_pred = best_cat_model.predict(X_test)
    print(f"\nPerformance for {model_name} Prediction using CatBoost:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R-squared:", r2_score(y_test, y_pred))


In [62]:
catBoost(X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg,"CatBoost")

Training CatBoost model for CatBoost prediction...

Best Parameters for CatBoost: {'depth': 9, 'iterations': 142, 'l2_leaf_reg': 2.8343478986616377, 'learning_rate': 0.16593820005455387, 'subsample': 0.7984250789732434}

Performance for CatBoost Prediction using CatBoost:
MAE: 1.1205526100236114
MSE: 124.79052113721175
R-squared: 0.769295021619468


In [63]:
catBoost(X_train_co, X_test_co, y_train_co, y_test_co,"CatBoost")

Training CatBoost model for CatBoost prediction...

Best Parameters for CatBoost: {'depth': 9, 'iterations': 113, 'l2_leaf_reg': 5.667628932479799, 'learning_rate': 0.18198808134726413, 'subsample': 0.8401537692938899}

Performance for CatBoost Prediction using CatBoost:
MAE: 0.6029920922945813
MSE: 30.94049847574257
R-squared: 0.8708881735560516
